# Profile analysis and degradation

Battery energy storage systems are capable of providing many useful services. But their great upfront investment cost means that the operative lifetime of the system greatly affects the overall profitability. This lifetime can vary greatly based on the cell characteristics and the operating conditions. 

In this notebook we will:
- Compare different battery storage system applications and their typical operating conditions.
- Explore the effects of these operating conditions on the lifetime of the system, based on the semi-empirical aging models of two cells.

In [ ]:
import numpy as np
import pandas as pd
import math

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# other plotting options
# import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
pd.options.plotting.backend = "plotly"
template = "plotly_white"

## Profile Analysis

The conceptual setup and data for our analysis are taken from the [*Standard Battery Application Profiles (SBAP) paper*](https://doi.org/10.1016/j.est.2019.101077). We take a look into 3 different storage systems, each serving their distinctive application:
- A grid scale storage system (1.6 MWh, 1.6 MW) that provides Frequency Containment Reserve (FCR).
- An industrial scale storage system (100 kWh, 40 kW) that performs Peak Shaving (PS).
- A home storage system (5kWh, 5kW) that aims to improve the self-consumption-increase (SCI) from a household through the *greedy*-strategy.

The operation of these storage systems was simulated with SimSES, resulting each in "power" and "soc" profiles. Further details can be found in the paper.


In [ ]:
# storage parameters - system size
fcr = dict(capacity=1.6e6, power=1.6e6)
ps  = dict(capacity=100e3, power=40e3)
sci = dict(capacity=5e3, power=5e3)

In [ ]:
df_fcr = pd.read_csv("../data/SBAP_FCR_profile.csv", index_col=0, parse_dates=True)
df_ps = pd.read_csv("../data/SBAP_PS_profile.csv", index_col=0, parse_dates=True)
df_sci = pd.read_csv("../data/SBAP_SCI_profile.csv", index_col=0, parse_dates=True)

In [ ]:
profiles = {
    "FCR": df_fcr,
    "PS": df_ps,
    "SCI": df_sci
}

In [ ]:
def plot_profiles(profiles):
    n = len(profiles)
    fig = make_subplots(rows=n, shared_xaxes=True, specs=n*[[{"secondary_y": True}]])
    fig.update_layout(template=template, height=800)

    for i, (name, df) in enumerate(profiles.items()):
        fig.add_trace(go.Scatter(x=df.index, y=df["power"], name=f"{name} power"), row=i+1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df["soc"], name=f"{name} soc"), row=i+1, col=1, secondary_y=True)
        fig.update_yaxes(title="Power / W")
        fig.update_yaxes(title="SOC", secondary_y=True)

    return fig

In [ ]:
plot_profiles(profiles)

Let's further examine the operating-profile characteristics of each application with some statistical analysis

<div class="alert alert-block alert-info">
<b>Task!</b> For each application:
<ul>
    <li> Calculate the E-rate profile and add it to their dataframe. </li>
    <li> Calculate the depth-of-cycle (DOC) profile and add it to their dataframe. </li>
    <li> Plot the histograms of the SOC, E-rate and DOC distribution. </li>
    <li> Calculate the following values:
        <ul>
            <li> Roundtrip efficiency. </li>
            <li> Full equivalent cycles (FEC). </li>
            <li> Average SOC. </li>
            <li> Temporal utilization. </li>
        </ul>
    </li>
</ul>
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b>
    <ul>
        <li>  Use the provided <code>calc_cycle_depth</code> function to calculate the DOC <i>of each timestep</i>. 
        Because a cycle depth can only be counted after the cycle has been completed, the value <code>0.0</code> is set for all the timesteps where a cycle is not yet complete. 
        <li> Use the <i>DOC-profile</i> to count the FECs.</li>
        <li> Make a 3x3 grid of subplots for the histograms:
        <ul>
          <li>  A row for each application </li>
          <li>  A column for each metric (SOC, E-rate, DOC) </li>
        </ul>
        </li>
        <li> Use your preferred plotting library: plotly, matplotlib or seaborn. </li>
        <li> Don't forget to omit the <i>0-depth</i> cycles for the DOC histogram-plot. </li>
    </ul>
</div>

In [ ]:
# depth of discharge cycle count
def calc_cycle_depth(df):
    soc = df["soc"].to_numpy()
    soc_diff = np.diff(soc)
    n = len(soc_diff)

    soc_sign_diff = np.diff(np.sign(soc_diff))
    soc_sign_diff = np.append(0, soc_sign_diff) # align with soc_diff
    nonzeros = np.nonzero(soc_sign_diff)[0] # find nonzeros
    start = np.append(0, nonzeros) 
    stop  = np.append(nonzeros, n) 

    x = len(start)
    soc_change = np.zeros(x)
    for i in range(x):
        soc_change[i] = np.sum(soc_diff[start[i]:stop[i]])

    start = start[soc_change > 0]
    soc_change = soc_change[soc_change > 0]

    idx = df.index[start]
    df.loc[idx, "doc"] = soc_change

    df.fillna(0.0, inplace=True)

    return

In [ ]:
# task


In [ ]:
# task


In [ ]:
# task


In [ ]:
# task


## Degradation

These applications have very different operational characteristics and present varied proportions of *stress factors*.
The cell-degradation can be accelerated (or slowed down) based on these stress factors, but the cell characteristics (chemistry, format, manufacturing) also play a key role.
<!-- For degradation or aging we refer here to the capacity loss caused by internal lithium-ion cell mechanism like SEI growth, loss of active material, lithium-plating and others. -->

We will assess the performance of 2 cells for each of these applications, these cells represent two of the most common cathode materials in lithium-ion batteries: Lithium-Iron-Phosphate (LFP) and Lithium-Nickel-Manganese-Cobalt-Oxide (NMC).
A common way to simulate the aging behavior of lithium-ion cells is through semi-empirical models. 
These are simple functions fitted from experimental data (lab tests of cells under varied stress conditions) 
to describe the relation between *stress factors* and the intensity of degradation rate.
This is in contrast to physics-based models, which although generally more accurate, are difficult to parametrize and computing intensive.

Semi-empirical models generally distinguish between calendar and cyclic degradation, under the assumption of superposition: the total degradation is the sum of the calendar and cyclic share.
Calendar degradation increases with the elapsed time and is strengthened according to specific stress factors (for example SOC and temperature), 
likewise cyclic degradation growths with the charge (or energy) throughput and is affected from cycling-related stress factors (for example DOC and C-rate).

### LFP-cell degradation model

The LFP-cell semi-empirical degradation models are based on the work of Nauman et al., published in the following 2 papers 
[[1]](https://doi.org/10.1016/j.est.2018.01.019) 
[[2]](https://doi.org/10.1016/j.jpowsour.2019.227666)

```
[1] Naumann, Maik, et al.
    "Analysis and modeling of calendar aging of a commercial LiFePO4/graphite cell."
    Journal of Energy Storage 17 (2018): 153-169.
    DOI: https://doi.org/10.1016/j.est.2018.01.019

[2] Naumann, Maik, Franz Spingler, and Andreas Jossen.
    "Analysis and modeling of cycle aging of a commercial LiFePO4/graphite cell."
    Journal of Power Sources 451 (2020): 227666.
    DOI: https://doi.org/10.1016/j.jpowsour.2019.227666
```
<!-- 
- Calendar degradation:
  - Square root dependency with elapsed time 
  - Stress factors: SOC and Temperature
  - Assumption constant Temperature of 25 °C
- Cyclic degradation
  - Square root dependency with charge throughput (in FEC)
  - Stress factors: DOC and C-rate (mean C-rate of cycle)
  - Assumptions we take the E-rate instead of C-rate 
-->



In [ ]:
def lfp_calendar(df, years=1):
    """Calendar-degradation model of LFP cell.
    
    Parameters
    ----------
    df: pandas-DataFrame
        Profile of storage system operation (1 year)
    year: int
        Number of years to simulate. Default 1 year
        
    Output
    ------
    loss_array: numpy-array
        Array with the capacity loss of each timestep in p.u. 
    """
    n = len(df)
    dt = (df.index[1] - df.index[0]).seconds
    soc_array = df_ps["soc"].to_numpy()

    C_QLOSS = 2.8575
    D_QLOSS = 0.60225
    
    cum_loss   = 0  # cumulative capacity loss in p.u.
    loss_array = np.zeros(len(df) * years) # capacity loss of each timestep in p.u.
    
    for year in range(years):
        for (i, soc) in enumerate(soc_array):
            # calculate stress factor dependent coefficients
            k_temp = 1.2571e-5 # temperature dependent coefficient (default: 25 °C)
            k_soc = C_QLOSS * (soc - 0.5) ** 3 + D_QLOSS # SOC dependent coefficient
            k = k_temp * k_soc

            # calculate capacity loss per step, based on virtual elapsed time and past total degradation.
            virtual_time = (cum_loss / k)**2 # virtual time in s
            loss = k_soc * k_temp * math.sqrt(virtual_time + dt) # total losses
            loss -=  cum_loss # time step losses
            loss_array[i + n*year] = loss
            
            # update variables
            cum_loss += loss  # increase cumulative losses

    return loss_array

In [ ]:
def lfp_cyclic(df, years=1):
    """Cyclic-degradation model of LFP cell.
    
    Parameters
    ----------
    df: pandas-DataFrame
        Profile of storage system operation (1 year)
    year: int
        Number of years to simulate. Default 1 year
        
    Output
    ------
    loss_array: numpy-array
        Array with the capacity loss of each timestep in p.u. 
    """
    n =len(df)
    
    # constants
    A_QLOSS = 0.0630
    B_QLOSS = 0.0971
    C_QLOSS = 4.0253
    D_QLOSS = 1.0923

    cum_loss = 0 # cumulative losses
    loss_array = np.zeros(n * years) # capacity loss of each timestep in p.u.
    idx_soc = []
        
    for year in range(years):
        for i, idx in enumerate(df.index):
            doc = df.loc[idx, "doc"] # depth of cycle in p.u.
            if doc != 0.0:
                # stress factors
                crate = df.loc[idx_soc, "erate"].mean() # average c-rate of cycle 1/h
                delta_fec = doc / 2 # 

                # calculate stress factor dependent coefficients
                k_c_rate = A_QLOSS * crate + B_QLOSS
                k_doc = C_QLOSS * (doc - 0.6)**3 + D_QLOSS

                # calculate capacity loss per step, based on virtual FEC and past total degradation.
                virtual_fec = (cum_loss * 100 / (k_c_rate * k_doc))**2
                loss = k_c_rate * k_doc * math.sqrt(virtual_fec + delta_fec) / 100  # total cyc. qloss in p.u.
                loss -= cum_loss  # relative qloss in pu in current timestep
                loss_array[i + n*year] = loss
                
                # update variables
                cum_loss += loss
                idx_soc = []    # reset index count
            else:
                idx_soc.append(idx)

    return loss_array

### NMC-cell degradation model

The NMC-cell semi-empirical models are based on the work of Schmalstieg et al., summarized in the following paper 
[[3]](https://doi.org/10.1016/j.jpowsour.2014.02.012)
```
[3] Schmalstieg, J., Käbitz, S., Ecker, M., & Sauer, D. U. (2014).
    A holistic aging model for Li (NiMnCo) O2 based 18650 lithium-ion batteries.
    Journal of Power Sources, 257, 325-334.
    DOI: https://doi.org/10.1016/j.jpowsour.2014.02.012
```

In [ ]:
def nmc_ocv(soc):
    """OCV-fitting function of NMC cell
    
    Parameter
    ---------
    soc: float
        SOC in p.u.
        
    Output
    ------
    ocv: flaot
        OCV voltage in V
    """
    k0 = 2.3885;  k1 = 2.1430;  k2 = -0.6287; k3 = -1.6708; k4 = 1.6161; k5 = 0.7234
    a1 = -7.7487; a2 = -0.0974; a3 = 1.2023;  a4 = 3.9977
    b1 = -0.1714; b2 = 2.6526

    ocv = k0 + \
            k1 / (1 + np.exp(a1 * (soc - b1))) + \
            k2 / (1 + np.exp(a2 * (soc - b2))) + \
            k3 / (1 + np.exp(a3 * (soc - 1))) +\
            k4 / (1 + np.exp(a4 * soc)) +\
            k5 * soc

    return ocv

In [ ]:
def nmc_calendar(df, years=1):
    """Calendar-degradation model of NMC cell.
    
    Parameters
    ----------
    df: pandas-DataFrame
        Profile of storage system operation (1 year)
    year: int
        Number of years to simulate. Default 1 year
        
    Output
    ------
    loss_array: numpy-array
        Array with the capacity loss of each timestep in p.u. 
    """
    dt = (df.index[1] - df.index[0]).seconds
    n = len(df)
    
    soc_array = df_ps["soc"].to_numpy()
    loss_array = np.zeros(n * years) # array with capacity loss of each timestep in p.u.
    time = 0 # total elapsed time

    for year in range(years):
        for (i, soc) in enumerate(soc_array):
            # stress factors
            voltage = nmc_ocv(soc) # convert SOC -> voltage in V
            temp = 25 + 273.15 # temperature in K
            
            # calculate stress factor dependent coefficients
            alpha_cap = (7.543 * voltage - 23.75)*1e6*math.exp(-6976/temp) 

            # calculate capacity loss per step, based on elapsed time and past total degradation.
            capacity_t0 = 1 - alpha_cap * (time / 86400) ** 0.75  # in p.u.
            capacity_t1 = 1 - alpha_cap * ((time + dt) / 86400) ** 0.75  # in p.u.
            loss = capacity_t0 - capacity_t1
            loss_array[i + n*year] = loss
            
            # update variables
            time += dt

    return loss_array

In [ ]:
# nmc - cyclic
def nmc_cyclic(df, years=1):
    """Cyclic-degradation model of NMC cell.
    
    Parameters
    ----------
    df: pandas-DataFrame
        Profile of storage system operation (1 year)
    year: int
        Number of years to simulate. Default 1 year
        
    Output
    ------
    loss_array: numpy-array
        Array with the capacity loss of each timestep in p.u. 
    """
    n = len(df)
    loss_array = np.zeros(n)

    qcell = 2.05
    throughput = 0
    idx_soc=[] # list of all indices between cycles
    loss_array = np.zeros(n * years)
    
    for year in range(years):
        for i, idx in enumerate(df.index):
            doc = df.loc[idx, "doc"]
            if doc != 0.0:
                # stress factors
                delta_throughput = qcell*doc
                soc_array = df.loc[idx_soc, "soc"]
                voltage_array = nmc_ocv(soc_array)
                voltage_mean = np.mean(voltage_array)
            
                # calculate stress factor dependent coefficients
                beta_cap = 7.348e-3*(voltage_mean - 3.667)**2 + 7.6e-4 + 4.081e-3*doc

                # calculate capacity loss per step, based on charge thorughput and past total degradation.
                capacity_t0 = 1 - beta_cap * math.sqrt(throughput)  # in p.u.
                capacity_t1 = 1 - beta_cap * math.sqrt(throughput + delta_throughput)  # in p.u.
                loss = capacity_t0 - capacity_t1
                loss_array[i + n*year] = loss
                
                # update variables
                throughput += delta_throughput
                idx_soc=[] # reset index count
            else:
                idx_soc.append(idx)

    return loss_array

<div class="alert alert-block alert-info">
<b>Task!</b> For all applications and both cells:
<ul>
    <li> Calculate the capacity loss profile. Take a 5-year operation horizon. </li>
    <li> Plot the estimated state-of-health (SOH) timeseries. </li>
    <li> Make a bar plot of the total capacity loss, differentiate between calendar and cyclic losses (stacked bar-plot). </li>
    <li> Comment on the cell performance:
        <ul>
            <li> Assuming an end-of-life (EOL) criterion of 80% SOH, which cell-application combinations will be still in deployment after the time horizon? </li>
            <li> What are the aging characteristic of each cell? Which cell is better suited for which application? </li>
        </ul>
    </li>
</ul>
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> 
Use the provided degradation models <code>lfp_calendar</code>, <code>lfp_cyclic</code>, <code>nmc_calendar</code> and <code>nmc_cyclic</code>. 
You don't have to focus on the implementation details of the models, only the interface.
</div>

In [ ]:
# task


In [ ]:
# task


In [ ]:
# task
